# Statistial Simulations

# Python Libraries

Python, like other programming languages, has an abundance of additional modules or libraries that augument the base framework and functionality of the language.

Think of a library as a collection of functions that can be accessed to complete certain programming tasks without having to write your own algorithm.

For this course, we will focus primarily on the following libraries:

* **Numpy** is a library for working with arrays of data.

* **Pandas** provides high-performance, easy-to-use data structures and data analysis tools.

* **Scipy** is a library of techniques for numerical and scientific computing.

* **Matplotlib** is a library for making graphs.

* **Seaborn** is a higher-level interface to Matplotlib that can be used to simplify many graphing tasks.

* **Statsmodels** is a library that implements many statistical techniques.

# Documentation

Reliable and accesible documentation is an absolute necessity when it comes to knowledge transfer of programming languages.  Luckily, python provides a significant amount of detailed documentation that explains the ins and outs of the language syntax, libraries, and more.  

Understanding how to read documentation is crucial for any programmer as it will serve as a fantastic resource when learning the intricacies of python.

Here is the link to the documentation of the python standard library: [Python Standard Library](https://docs.python.org/3/library/index.html#library-index)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels as stat

1.ads bid 如果有两个bidder X AND Y， the bid amount is uniform[0,1] distribution. what is the expected revenue?
what if three bidders?


In [20]:

# two bidders
x = np.random.uniform(low=0,high=1,size=10000)
y = np.random.uniform(0,1,size=10000)
z = np.random.uniform(0,1,size=10000)
xyz = np.stack((x,y,z),axis=0)
xyz_sort = np.sort(xyz,axis=0)
exp1 = np.mean(np.maximum(x,y))## maximum take two ndarrys where max take one ndarray
exp2 = np.mean(np.minimum(x,y))
exp3 = np.mean(np.minimum(x,y,z))

2. 非常简单的for loop coding 题。用R或者python写都可以。计算 sqrt(1) + sqrt(2) + ... + sqrt(100)

In [25]:
import math as mt
sum([mt.sqrt(i) for i in range(1,101)])
sum([np.sqrt(i) for i in range(1,101)])

671.4629471031477

3. generate poisson distribution using unifrom
https://www.johndcook.com/blog/2010/06/14/generating-poisson-random-values/

In [28]:
def poisson(lam):
    k, p = 0, 1
    while p > mt.exp(-lam):
        k += 1
        p = p*np.random.uniform()
    return k

In [42]:
poisson(2)

1

# Sampling

In [3]:
# bootstrapping
# Draw some random sample with replacement and append mean to mean_lengths.
mean_lengths, sims = [], 1000
wrench_lengths = np.random.uniform(size=1000)
for i in range(sims):
    temp_sample = np.random.choice(wrench_lengths, replace=True, size=len(wrench_lengths))
    sample_mean = np.mean(temp_sample)
    mean_lengths.append(sample_mean)
    
# Calculate bootstrapped mean and 95% confidence interval.
boot_mean = np.mean(mean_lengths)
boot_95_ci = np.percentile(mean_lengths, [2.5, 97.5])
print("Bootstrapped Mean Length = {}, 95% CI = {}".format(boot_mean, boot_95_ci))

Bootstrapped Mean Length = 0.4909181528776206, 95% CI = [0.47222953 0.50840311]


Non-standard estimators
In the last exercise, you ran a simple bootstrap that we will now modify for more complicated estimators.

Suppose you are studying the health of students. You are given the height and weight of 1000 students and are interested in the median height as well as the correlation between height and weight and the associated 95% CI for these quantities. Let's use bootstrapping.

Examine the pandas DataFrame df with the heights and weights of 1000 students. Using this, calculate the 95% CI for both the median height as well as the correlation between height and weight.

In [6]:
df.columns

Index(['ID', 'Age', 'Gender', 'GenderGroup', 'Glasses', 'GlassesGroup',
       'Height', 'Wingspan', 'CWDistance', 'Complete', 'CompleteGroup',
       'Score'],
      dtype='object')

In [10]:
# Sample with replacement and calculate quantities of interest
# Store the url string that hosts our .csv file
url = "Cartwheeldata.csv"

# Read the .csv file and store it as a pandas Data Frame
df = pd.read_csv(url)

sims, data_size, height_medians, hw_corr = 1000, df.shape[0], [], []
for i in range(sims):
    tmp_df = df.sample(n=data_size,replace=True)
    height_medians.append(tmp_df.median().Height)
    hw_corr.append(tmp_df.corr().Height)

# Calculate confidence intervals
height_median_ci = np.percentile(height_medians,[2.5,97.5])
height_weight_corr_ci = np.percentile(hw_corr,[2.5,97.5])
print("Height Median CI = {} \nHeight Weight Correlation CI = {}".format( height_median_ci, height_weight_corr_ci))

Height Median CI = [65. 70.] 
Height Weight Correlation CI = [-0.55614018  1.        ]


Bootstrapping regression
Now let's see how bootstrapping works with regression. Bootstrapping helps estimate the uncertainty of non-standard estimators. Consider the R2 statistic associated with a regression. When you run a simple least squares regression, you get a value for R2. But let's see how can we get a 95% CI for R2.

Examine the DataFrame df with a dependent variable y and two independent variables X1 and X2 using df.head(). We've already fit this regression with statsmodels (sm) using:

reg_fit = sm.OLS(df['y'], df.iloc[:,1:]).fit()
Examine the result using reg_fit.summary() to find that R2=0.3504. Use bootstrapping to calculate the 95% CI.

In [18]:
import statsmodels.api as sm
import numpy as np
rsquared_boot, coefs_boot, sims = [], [], 1000
reg_fit = sm.OLS(df['Height'], df.iloc[:,6]).fit()

# Run 1K iterations
for i in range(sims):
    # First create a bootstrap sample with replacement with n=df.shape[0]
    bootstrap = df.sample(n=df.shape[0],replace=True)
    # Fit the regression and append the r square to rsquared_boot
    rsquared_boot.append(sm.OLS(bootstrap['Height'],bootstrap.iloc[:,6]).fit().rsquared)

# Calculate 95% CI on rsquared_boot
r_sq_95_ci = np.percentile(rsquared_boot,[2.5,97.5])
r_sq_95_ci

array([1., 1.])

In the next few exercises, we will run a significance test using permutation testing. 
Hypothesis testing - Difference of means
We want to test the hypothesis that there is a difference in the average donations received from A and B. Previously, you learned how to generate one permutation of the data. Now, we will generate a null distribution of the difference in means and then calculate the p-value.

For the null distribution, we first generate multiple permuted datasets and store the difference in means for each case. We then calculate the test statistic as the difference in means with the original dataset. Finally, we calculate the p-value as twice the fraction of cases where the difference is greater than or equal to the absolute value of the test statistic (2-sided hypothesis). A p-value of less than say 0.05 could then determine statistical significance.

In [20]:
# Generate permutations equal to the number of repetitions
data = np.random.uniform(size = 2000)
donations_A = range(100)
donations_B = range(100)
reps = 100
perm = np.array([np.random.permutation(len(donations_A) + len(donations_B)) for i in range(reps)])
permuted_A_datasets = data[perm[:, :len(donations_A)]]
permuted_B_datasets = data[perm[:, len(donations_A):]]

# Calculate the difference in means for each of the datasets
samples = np.mean(permuted_A_datasets,axis=1) - np.mean(permuted_B_datasets,axis=1)

# Calculate the test statistic and p-value
test_stat = np.mean(donations_A) - np.mean(donations_B)
p_val = 2*np.sum(samples >= np.abs(test_stat))/reps
print("p-value = {}".format(p_val))

p-value = 0.78


In [25]:
type(permuted_A_datasets)

numpy.ndarray

Hypothesis testing - Non-standard statistics
In the previous two exercises, we ran a permutation test for the difference in mean values. Now let's look at non-standard statistics.

Suppose that you're interested in understanding the distribution of the donations received from websites A and B. For this, you want to see if there's a statistically significant difference in the median and the 80th percentile of the donations. Permutation testing gives you a wonderfully flexible framework for attacking such problems.

Let's go through running a test to see if there's a difference in the median and the 80th percentile of the distribution of donations. As before, you're given the donations from the websites A and B in the variables donations_A and donations_B respectively.

In [26]:
# Calculate the difference in 80th percentile and median for each of the permuted datasets (A and B)
samples_percentile = np.percentile(permuted_A_datasets, 80, axis=1) - np.percentile(permuted_B_datasets, 80, axis=1)
samples_median = np.median(permuted_A_datasets, axis=1) - np.median(permuted_B_datasets, axis=1)

# Calculate the test statistic from the original dataset and corresponding p-values
test_stat_percentile = np.percentile(donations_A, 80) - np.percentile(donations_B, 80)
test_stat_median = np.median(donations_A) - np.median(donations_B)
p_val_percentile = 2*np.sum(samples_percentile >= np.abs(test_stat_percentile))/reps
p_val_median = 2*np.sum(samples_median >= np.abs(test_stat_median))/reps

print("80th Percentile: test statistic = {}, p-value = {}".format(test_stat_percentile, p_val_percentile))
print("Median: test statistic = {}, p-value = {}".format(test_stat_median, p_val_median))

80th Percentile: test statistic = 0.0, p-value = 0.88
Median: test statistic = 0.0, p-value = 0.88


Power Analysis - Part II
Previously, we simulated one instance of the experiment & generated a p-value. We will now use this framework to calculate statistical power. Power of an experiment is the experiment's ability to detect a difference between treatment & control if the difference really exists. It's good statistical hygiene to strive for 80% power.

For our website, we want to know how many people need to visit each variant, such that we can detect a 10% increase in time spent with 80% power. For this, we start with a small sample (50), simulate multiple instances of this experiment & check power. If 80% power is reached, we stop. If not, we increase the sample size & try again.

# statsmodels
https://www.statsmodels.org/stable/_modules/statsmodels/stats/power.html

In [33]:
import scipy.stats as st
sample_size = 50
control_mean = 1
control_sd = 1
effect_size = 0.1
sim = 100
# Keep incrementing sample size by 10 till we reach required power
while 1:
    control_time_spent = np.random.normal(loc=control_mean, scale=control_sd, size=(sample_size,sims))
    treatment_time_spent = np.random.normal(loc=control_mean*(1+effect_size), scale=control_sd, size=(sample_size,sims))
    t, p = st.ttest_ind(treatment_time_spent, control_time_spent)
    
    # Power is the fraction of times in the simulation when the p-value was less than 0.05
    power = (p < 0.05).sum()/sims
    if power > 0.8: 
        break;
    else: 
        sample_size += 10
print("For 80% power, sample size required = {}".format(sample_size))

For 80% power, sample size required = 1510


This is a simple exercise introducing the concept of Monte Carlo Integration.

Here we will evaluate a simple integral ∫10xexdx. We know that the exact answer is 1, but simulation will give us an approximate solution, so we can expect an answer close to 1. As we saw in the video, it's a simple process. For a function of a single variable f(x):

Get the limits of the x-axis (xmin,xmax) and y-axis (max(f(x)),min(min(f(x)),0)).
Generate a number of uniformly distributed point in this box.
Multiply the area of the box ((max(f(x)−min(f(x))×(xmax−xmin)) by the fraction of points that lie below f(x).
Upon completion, you will have a framework for handling definite integrals using Monte Carlo Integration.

In [35]:
# Define the sim_integrate function
def sim_integrate(func, xmin, xmax, sims):
    x = np.random.uniform(xmin, xmax, sims)
    y = np.random.uniform(min(min(func(x)),0), max(func(x)), sims)
    area = (max(y) - min(y))*(xmax-xmin)
    result = area * sum( (y-np.array(func(x)))<=0 )/sims
    return result
    
# Call the sim_integrate function and print results
result = sim_integrate(func = lambda x: x*np.exp(x), xmin = 0, xmax = 1, sims = 500)
print("Simulated answer = {}, Actual Answer = 1".format(result))

Simulated answer = 0.9788947624753406, Actual Answer = 1


Calculating the value of pi
Now we work through a classic example - estimating the value of π.

Imagine a square of side 2 with the origin (0,0) as its center and the four corners having coordinates (1,1),(1,−1),(−1,1),(−1,−1). The area of this square is 2×2=4. Now imagine a circle of radius 1 with its center at the origin fitting perfectly inside this square. The area of the circle will be π×radius2=π.

To estimate π, we randomly sample multiple points in this square & get the fraction of points inside the circle (x2+y2<=1). The area of the circle then is 4 times this fraction, which gives us our estimate of π.

After this exercise, you'll have a grasp of how to use simulation for computation.

In [36]:
# Initialize sims and circle_points
sims, circle_points = 10000, 0

for i in range(sims):
    # Generate the two coordinates of a point
    point = np.random.uniform(-1,1,size=2)
    # if the point lies within the unit circle, increment counter
    within_circle = point[0]**2 + point[1]**2 <= 1
    if within_circle == True:
        circle_points +=1
        
# Estimate pi as 4 times the avg number of points in the circle.
pi_sim = 4*circle_points/sims
print("Simulated value of pi = {}".format(pi_sim))

Simulated value of pi = 3.1284
